In [54]:
import sys, os

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(ROOT, "src"))

from dagc.data.dataset_gnn import GraphDirectoryDataset

train_dir = os.path.join(ROOT, "data", "synthetic_graphs", "ba", "train")
dataset = GraphDirectoryDataset(train_dir, device="cpu")

print(len(dataset))
sample = dataset[0]
print(sample.graph.number_of_nodes())
print(sample.x.shape, sample.edge_index.shape)


120
1000
torch.Size([1000, 2]) torch.Size([2, 5982])


In [55]:
from dagc.sparsifiers.gnn.train import train_gnn_sparsifier_on_dataset

model = train_gnn_sparsifier_on_dataset(
    train_dir="../data/synthetic_graphs/ba/train",
    keep_ratio=0.8,
    num_steps=2,
    num_epochs=10,
    device="cpu",
)


[Epoch 1/10] avg_loss = 0.838764 over 120 graphs
[Epoch 2/10] avg_loss = 0.927309 over 120 graphs
[Epoch 3/10] avg_loss = 0.965678 over 120 graphs
[Epoch 4/10] avg_loss = 1.021908 over 120 graphs
[Epoch 5/10] avg_loss = 1.079299 over 120 graphs
[Epoch 6/10] avg_loss = 1.138908 over 120 graphs
[Epoch 7/10] avg_loss = 1.194282 over 120 graphs
[Epoch 8/10] avg_loss = 1.235830 over 120 graphs
[Epoch 9/10] avg_loss = 1.252502 over 120 graphs
[Epoch 10/10] avg_loss = 1.258706 over 120 graphs


In [56]:
import os
import sys
from pathlib import Path

import torch
import networkx as nx

# Point ROOT to repo root (assuming notebook is in `scripts/`)
ROOT = Path(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(str(ROOT / "src"))

from dagc.data.graph_loading import load_graphs_from_dir, graph_to_tensors
from dagc.graphs.seeds import generate_seed_set
from dagc.sparsifiers.classical.random_edge import random_edge_sparsify
from dagc.sparsifiers.gnn.gnn_sparsifier import GNNSparsifier
from dagc.sparsifiers.gnn.utils import sparsify_graph_with_model
from dagc.diffusion.ic_experiment import run_ic_experiment
from dagc.metrics.diffusion_metrics import (
    compare_diffusion_summaries,
    topk_jaccard_from_summaries,
    spread_difference,
)


In [57]:
# Change to "train" if you want the train set instead
graph_dir = ROOT / "data" / "synthetic_graphs" / "ba" / "test"

graphs = load_graphs_from_dir(str(graph_dir))
len(graphs)

40

In [58]:
graphs_10 = graphs[:10]
print("Loaded", len(graphs_10), "graphs for evaluation")

Loaded 10 graphs for evaluation


In [59]:
def evaluate_one_graph(
    G: nx.Graph,
    model: GNNSparsifier,
    keep_ratio: float = 0.5,
    seed_set_size: int = 10,
    activation_prob: float = 0.1,
    num_runs: int = 200,
    base_seed: int = 12345,
    device: str = "cpu",
):
    """
    For a single graph G:
      - choose a seed set
      - run IC on original G
      - run IC on random-sparsified H_rand
      - run IC on GNN-sparsified H_gnn
      - compute metrics for each vs original

    Returns:
      metrics_rand, metrics_gnn,
      orig_summary, rand_summary, gnn_summary,
      H_rand, H_gnn
    """
    n = G.number_of_nodes()

    # 1) Seed set (same seeds for all variants)
    seed_set = generate_seed_set(G, k=seed_set_size, strategy="high_degree")

    # 2) Original graph IC summary
    orig_summary = run_ic_experiment(
        graph=G,
        seed_set=seed_set,
        num_runs=num_runs,
        activation_prob=activation_prob,
        base_seed=base_seed,
    )

    # 3) Random sparsifier
    rand_comp = random_edge_sparsify(
        graph=G,
        keep_ratio=keep_ratio,
        rng_seed=base_seed + 999,
    )
    H_rand = rand_comp.graph

    rand_summary = run_ic_experiment(
        graph=H_rand,
        seed_set=seed_set,
        num_runs=num_runs,
        activation_prob=activation_prob,
        base_seed=base_seed + 1000,
    )

    # 4) GNN sparsifier
    H_gnn = sparsify_graph_with_model(
        model=model,
        G=G,
        keep_ratio=keep_ratio,
        device=device,
    )

    gnn_summary = run_ic_experiment(
        graph=H_gnn,
        seed_set=seed_set,
        num_runs=num_runs,
        activation_prob=activation_prob,
        base_seed=base_seed + 2000,
    )

    # 5) Metrics vs original
    metrics_rand = compare_diffusion_summaries(
        original=orig_summary,
        compressed=rand_summary,
    )
    metrics_gnn = compare_diffusion_summaries(
        original=orig_summary,
        compressed=gnn_summary,
    )

    # Add normalized spread difference / |V|
    metrics_rand["spread_rel_diff_by_V"] = spread_difference(
        spread_original=orig_summary.expected_spread,
        spread_compressed=rand_summary.expected_spread,
        num_nodes_original=n,
    )
    metrics_gnn["spread_rel_diff_by_V"] = spread_difference(
        spread_original=orig_summary.expected_spread,
        spread_compressed=gnn_summary.expected_spread,
        num_nodes_original=n,
    )

    # Add top-k Jaccards of "most activated nodes"
    for k in [10, 20, 50]:
        if k > n:
            break
        metrics_rand[f"top{k}_jacc"] = topk_jaccard_from_summaries(
            original=orig_summary,
            compressed=rand_summary,
            k=k,
        )
        metrics_gnn[f"top{k}_jacc"] = topk_jaccard_from_summaries(
            original=orig_summary,
            compressed=gnn_summary,
            k=k,
        )

    return (
        metrics_rand,
        metrics_gnn,
        orig_summary,
        rand_summary,
        gnn_summary,
        H_rand,
        H_gnn,
    )


In [62]:
keep_ratio = 0.3
seed_set_size = 10
activation_prob = 0.1
num_runs = 200

all_rand = []
all_gnn = []

for idx, G in enumerate(graphs_10):
    print(f"\n================  Graph {idx}  ================")
    print(f"|V| = {G.number_of_nodes()}, |E| = {G.number_of_edges()}")

    (
        metrics_rand,
        metrics_gnn,
        orig_summary,
        rand_summary,
        gnn_summary,
        H_rand,
        H_gnn,
    ) = evaluate_one_graph(
        G=G,
        model=model,
        keep_ratio=keep_ratio,
        seed_set_size=seed_set_size,
        activation_prob=activation_prob,
        num_runs=num_runs,
        base_seed=12345 + idx * 10000,  # different seed per graph
        device="cpu",
    )

    all_rand.append(metrics_rand)
    all_gnn.append(metrics_gnn)

    # Edge counts
    print(f"|E_rand| = {H_rand.number_of_edges()} (random sparsifier)")
    print(f"|E_gnn|  = {H_gnn.number_of_edges()} (GNN sparsifier)")

    # Raw spreads
    orig_spread = orig_summary.expected_spread
    rand_spread = rand_summary.expected_spread
    gnn_spread = gnn_summary.expected_spread

    print("\nRaw expected spreads:")
    print(f"  Original:       {orig_spread:.3f}")
    print(f"  Random sparse:  {rand_spread:.3f}")
    print(f"  GNN sparse:     {gnn_spread:.3f}")

    print("\nRandom sparsifier vs original:")
    signed_diff_rand = metrics_rand["spread_abs_diff"]
    print(f"  spread_diff (signed):  {signed_diff_rand:.3f}")
    print(f"  spread_diff (abs):     {abs(signed_diff_rand):.3f}")
    print(f"  spread_rel_error:      {metrics_rand['spread_rel_error']:.4f}")
    print(f"  spread_rel_diff_by_V:  {metrics_rand['spread_rel_diff_by_V']:.4f}")
    print(f"  prob_mse:              {metrics_rand['prob_mse']:.6f}")
    print(f"  prob_mae:              {metrics_rand['prob_mae']:.6f}")
    for k in [10, 20, 50]:
        key = f"top{k}_jacc"
        if key in metrics_rand:
            print(f"  top-{k} Jaccard:        {metrics_rand[key]:.3f}")

    print("\nGNN sparsifier vs original:")
    signed_diff_gnn = metrics_gnn["spread_abs_diff"]
    print(f"  spread_diff (signed):  {signed_diff_gnn:.3f}")
    print(f"  spread_diff (abs):     {abs(signed_diff_gnn):.3f}")
    print(f"  spread_rel_error:      {metrics_gnn['spread_rel_error']:.4f}")
    print(f"  spread_rel_diff_by_V:  {metrics_gnn['spread_rel_diff_by_V']:.4f}")
    print(f"  prob_mse:              {metrics_gnn['prob_mse']:.6f}")
    print(f"  prob_mae:              {metrics_gnn['prob_mae']:.6f}")
    for k in [10, 20, 50]:
        key = f"top{k}_jacc"
        if key in metrics_gnn:
            print(f"  top-{k} Jaccard:        {metrics_gnn[key]:.3f}")



================  Graph 0  ================
|V| = 1000, |E| = 2991
|E_rand| = 999 (random sparsifier)
|E_gnn|  = 897 (GNN sparsifier)

Raw expected spreads:
  Original:       129.765
  Random sparse:  28.105
  GNN sparse:     77.405

Random sparsifier vs original:
  spread_diff (signed):  -101.660
  spread_diff (abs):     101.660
  spread_rel_error:      0.7834
  spread_rel_diff_by_V:  0.1017
  prob_mse:              0.016012
  prob_mae:              0.102050
  top-10 Jaccard:        1.000
  top-20 Jaccard:        0.333
  top-50 Jaccard:        0.176

GNN sparsifier vs original:
  spread_diff (signed):  -52.360
  spread_diff (abs):     52.360
  spread_rel_error:      0.4035
  spread_rel_diff_by_V:  0.0524
  prob_mse:              0.004291
  prob_mae:              0.055970
  top-10 Jaccard:        1.000
  top-20 Jaccard:        0.739
  top-50 Jaccard:        0.613

================  Graph 1  ================
|V| = 1000, |E| = 2991
|E_rand| = 999 (random sparsifier)
|E_gnn|  = 897 (GNN 